## System test

Let's check if the current user has admin rights or included into the 'video' group

In [1]:
!id

uid=1000(jovyan) gid=100(users) groups=100(users),0(root),27(sudo),44(video),106(i2c)


In [2]:
!cat /etc/group

root:x:0:
daemon:x:1:
bin:x:2:
sys:x:3:
adm:x:4:
tty:x:5:
disk:x:6:
lp:x:7:
mail:x:8:
news:x:9:
uucp:x:10:
man:x:12:
proxy:x:13:
kmem:x:15:
dialout:x:20:
fax:x:21:
voice:x:22:
cdrom:x:24:
floppy:x:25:
tape:x:26:
sudo:x:27:
audio:x:29:
dip:x:30:
www-data:x:33:
backup:x:34:
operator:x:37:
list:x:38:
irc:x:39:
src:x:40:
gnats:x:41:
shadow:x:42:
utmp:x:43:
video:x:44:
sasl:x:45:
plugdev:x:46:
staff:x:50:
games:x:60:
users:x:100:
nogroup:x:65534:
_ssh:x:101:
input:x:102:
sgx:x:103:
kvm:x:104:
render:x:105:
i2c:x:106:


List video devices available for Jupyter

In [3]:
ls /dev/video*

/dev/video10  /dev/video15  /dev/video18  /dev/video21
/dev/video13  /dev/video16  /dev/video19  /dev/video23
/dev/video14  /dev/video17  /dev/video20  /dev/videosync


## Video test

To read camera image the next algorithm can be used:

```python
# Activate camera object
cap = cv2.VideoCapture(0)

# Take a picture
_, frame = cap.read()

# Convert the picture to bytes
_, frame = cv2.imencode('.png', frame)
frame = frame.tobytes()

# Release the camera object
cap.release()```


In [6]:
import solara, cv2
from time import sleep

@solara.component
def Page(): 
    frame_react = solara.use_reactive(None)
    video_running_react = solara.use_reactive(False)
    cap_react = solara.use_reactive(None)
    
    def change_state():
        if not video_running_react.value:
            cap_react.value = cv2.VideoCapture(0)
            video_running_react.value = True
        else:
            video_running_react.value = False
            sleep(1)
            frame_react.value = None
            cap_react.value.release()
            
    def show_image():
        while True:
            if video_running_react.value and cap_react.value is not None:
                frame = cv2.flip(cap_react.value.read()[1], 1)
                frame_react.value = cv2.imencode('.png', frame)[1].tobytes()
    
    with solara.Card(title="Robot video"):
        solara.Button(
            label = 'Stop Video' if video_running_react.value else 'Start Video', 
                on_click = change_state)
        if frame_react.value is not None:
            solara.Image(frame_react.value) 

    solara.use_thread(show_image)

Page()

Cannot show ipywidgets in text

In [13]:
import gpiod
chip = gpiod.chip.open("/dev/gpiochip0")
#chip0 = gpiod.Chip("/dev/i2c-0")
#print(chip0)
#i2c = chip0.get_i2c_device(0x40)
#i2c.transfer([gpiod.Message([0x00] + 500)])

AttributeError: module 'gpiod.chip' has no attribute 'open'